# Amazon Bedrock으로 RAG 적용된 Chatbot 만들기
#### 목표 : 한글이 지원되는 Claude-v2 model로 RAG 기반 챗봇 만들기

> *이 노트북은 SageMaker Studio 커널 **`Data Science 3.0`** 에서 잘 동작합니다.*

### Amazon Bedrock과 Langchain 기술을 활용하는 챗봇

![chatbot_bedrock](./img/chatbot_bedrock.png)

### Amazon Bedrock으로 챗봇을 구축하기 위한 랭체인 프레임워크
- 챗봇과 같은 대화형 인터페이스에서는 단기적 수준뿐만 아니라 장기적 수준에서도 이전 상호 작용을 기억하는 것이 매우 중요합니다.
- LangChain은 메모리 구성 요소를 두 가지 형태로 제공합니다.먼저 LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미 유틸리티를 제공합니다.모듈식으로 설계되어 사용 방식에 관계없이 유용하게 사용할 수 있습니다.둘째, LangChain은 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다. 이를 통해 다양한 유형의 추상화를 쉽게 정의하고 상호 작용할 수 있으므로 강력한 챗봇을 쉽게 구축할 수 있습니다.
- RAG(Retrief-Augmented Generation)를 통해 외부 데이터로 모델의 기초 데이터를 보완합니다. LangChain의 Conversational RetrieveChain 클래스를 사용하여 챗봇과 RAG 기능을 한 번의 통화로 결합합니다.

### 컨텍스트를 활용한 챗봇 구축하기 - 핵심 요소

컨텍스트 인식 챗봇을 구축하는 첫 번째 프로세스는 컨텍스트에 대한임베딩을 생성하는 것입니다.일반적으로 임베딩 모델을 통해 실행되고 일종의 벡터 저장소에 저장될 임베딩을 생성하는 통합 프로세스가 있습니다. 이 예제에서는 이를 위해 Amazon Titan 임베딩 모델을 사용하고 있습니다.

![embeddings_lang](./img/embeddings_lang.png)

두 번째 프로세스는 사용자 요청 오케스트레이션, 상호 작용, 호출 및 결과 반환입니다.

![chatbot_lang](./img/chatbot_lang.png)

### 목표 아키텍처 

### Prerequisites
- 몇 가지 추가 종속성이 필요합니다.
> FAISS : 벡터 임베딩 저장용 <br>
> iPyWidgets : 노트북의 대화형 UI 위젯용 <br>
> PyPDF : PDF 파일 처리용

In [2]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.308 "pypdf>=3.8,<4" sqlalchemy==2.0.21

Note: you may need to restart the kernel to use updated packages.


## 1. Amazon Bedrock Client 생성

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## 2. 챗봇 (기본 - 컨텍스트 없음)

Anthropic의 Claude-v2 모델을 사용하여 한글이 지원되는 챗봇을 만들 것 입니다.

우리는 LangChain의 [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain)을 사용하여 시작합니다. 또한 메시지 저장을 위해 [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html)를 사용합니다. 메시지 목록으로 기록을 얻을 수도 있습니다(채팅 모델에서 매우 유용합니다).

챗봇은 이전 상호작용을 기억해야 합니다. 대화 기억을 통해 우리는 그렇게 할 수 있습니다. 대화형 메모리를 구현하는 방법에는 여러 가지가 있습니다. LangChain의 맥락에서 이들은 모두 ConversationChain 위에 구축됩니다.



In [4]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory

cl_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1024,
                  "temperature": 0.1,
                  "top_k": 250,
                  "top_p": 1.0
                 },
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="안녕하세요?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요?
AI:

> Finished chain.
 네, 안녕하세요!


### 결과 분석
[효과적인 Claude 프롬프트](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)는  `\n\nHuman: 텍스트 \n\nAassistant:` 형식이어야 합니다.

Claude의 프롬프트 작성 방법에 대해 자세히 알아보려면 [Anthropic 문서](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)를 확인하세요.

In [5]:
# boto3 및 langchain 버전 확인
# 2023년 10월 5일 기준 langchain 버전은 0.0.304 입니다.

import langchain

print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.308
boto3 version check: 1.28.60


## 3. 프롬프트 템플릿(Langchain)을 이용한 챗봇

LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 기능을 제공합니다.

[PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) 클래스를 사용하여 f-string 템플릿에서 프롬프트를 구성하겠습니다.

In [6]:
from langchain import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude

claude_prompt = PromptTemplate.from_template("""다음은 사람과 인공지능의 친근한 대화입니다. 인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
{history}

Human: {input}

Assistant:
""")


print("claude_prompt: \n", claude_prompt)

claude_prompt: 
 input_variables=['history', 'input'] template='다음은 사람과 인공지능의 친근한 대화입니다. 인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.\n\nCurrent conversation:\n{history}\n\nHuman: {input}\n\nAssistant:\n'


In [7]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

chat_memory = ConversationBufferMemory(human_prefix='Human', ai_prefix='Assistant')
conversation = ConversationChain(llm=cl_llm, verbose=False, memory=chat_memory)

In [8]:
conversation.prompt = claude_prompt

print_ww(conversation.predict(input="안녕하세요?"))

 안녕하세요! 반갑습니다.


#### (1) 새로운 질문

모델이 초기 메시지로 응답했습니다. 몇 가지 질문을 해보겠습니다.

In [9]:
print_ww(conversation.predict(input="새로운 정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요."))

 네, 새로운 정원을 시작하는 몇 가지 팁을 알려드리겠습니다.

먼저, 정원의 위치를 잘 선택하는 것이 중요합니다. 해가 잘 드는 곳으로 선택하시는 것이 좋습니다. 또한 물 공급이 용이한 곳을 고르세요.

다음으로 흙의 상태를 확인하세요. 흙이 너무 건조하거나 점토질이면 퇴비를 추가로 넣어주는 것이 좋습니다. pH 수준도 확인하세요.

정원 배치를 계획할 때에는 저온성 작물과 고온성 작물을 구분하여 심는 것이 중요합니다. 또한 키가 큰 식물과 작은 식물의 배치도 중요합니다.

물주기와 관수 방법도 식물에 따라 다르므로 잘 확인하세요. 일정한 시기에 관수와 배수를 해주는 것이 식물 성장에 도움이 됩니다.

마지막으로, 해충 방지와 병충해 관리도 철저히 하는 것이 좋습니다. 이를 통해 건강한 정원을 가꿀 수 있습니다.

정원 가꾸기에 관심이 많으시면 책이나 온라인을 통해 더 많은 정보를 찾아보시기 바랍니다. 즐거운 정원 가꾸기 되세요!


#### (2) 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문해 보겠습니다.

In [10]:
print_ww(conversation.predict(input="좋아요. 토마토에도 어울릴까요?"))

 네, 제가 말씀드린 팁들은 토마토 정원을 가꾸는 데도 적용할 수 있습니다.

토마토는 일교차가 크고 햇볕이 잘 드는 곳을 좋아하므로 그런 곳을 정원 위치로 선택하세요. 또한 토마토는 물이 잘 빠지는 토양을 선호하므로 배수가 잘 되는 곳이 좋습니다.

토마토를 심기 전에 흙의 pH를 확인하고 필요하다면 석회나 퇴비를 추가로 넣어주세요. 토마토는 약산성 토양을 선호합니다.

토마토는 성장 형태에 따라 결실기가 짧은 종류와 긴 종류가 있는데, 이를 고려하여 정원 내 배치를 계획하세요.

물주기는 토마토 열매가 익기 시작하면 조금 줄이고, 열매가 익으면 물을 주지 않는 것이 좋습니다.

이외에도 병해충 방지, 지주 설치, 시비 등을 철저히 하시면 토마토 정원 가꾸기에 도움이 될 것입니다.


#### (3) 대화 마무리

In [11]:
print_ww(conversation.predict(input="그게 다야, 고마워!"))

 네, 처음 새로운 정원을 시작하시는 것이다보니 정보가 많이 필요하실 것 같습니다. 제가 알려드린 내용이 새로운 정원 가꾸기에 도움이 되었기를 바랍니다. 더 궁금한 점이 있으시면
언제든지 문의주세요. 즐거운 정원 가꾸기 되세요!


## 4. ipywidgets를 사용한 대화형 세션

다음 유틸리티 클래스를 사용하면 Claude와 보다 자연스러운 방식으로 상호 작용할 수 있습니다. 입력창에 질문을 적고 Claude의 답변을 받습니다. 그렇게 대화를 이어갈 수 있습니다.

In [12]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

#### (1) 채팅을 시작해 보겠습니다. 다음 질문을 테스트할 수도 있습니다.

1. 농담 하나 해줘
2. 또 다른 농담을 들려주세요
3. 첫 번째 농담은 무엇이었나요?
4. 첫 번째 농담과 같은 주제로 또 다른 농담을 할 수 있나요?

위의 4가지를 순서대로 아래에 입력하시고, "Send" 버튼을 눌러 보세요.

In [13]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

## 5. 페르소나를 활용한 챗봇
- skip

## 6. 맥락을 가진 챗봇 (RAG)

이 사용 사례에서는 Chatbot에게 이전에 본 적이 없는 외부 코퍼스의 질문에 답변하도록 요청합니다. 이를 위해 RAG(Retrieval Augmented Generation)라는 패턴을 적용합니다. 아이디어는 말뭉치를 덩어리로 인덱싱한 다음 덩어리와 질문 사이의 의미론적 유사성을 사용하여 말뭉치의 어느 섹션이 답변을 제공하는 데 관련될 수 있는지 찾는 것입니다. 마지막으로 가장 관련성이 높은 청크가 집계되어 기록을 제공하는 것과 유사하게 ConversationChain에 컨텍스트로 전달됩니다.

<b>Titan Embeddings Model</b>을 사용하여 벡터를 생성하겠습니다. 그런 다음 이 벡터는 메모리 내 벡터 데이터 저장소를 제공하는 오픈 소스 라이브러리인 FAISS에 저장됩니다. 챗봇이 질문을 받으면 FAISS에 질문을 쿼리하고 의미상 가장 가까운 텍스트를 검색합니다. 이것이 우리의 대답이 될 것입니다.

### 6.1 2022년 아마존 주주 서한 pdf 문서로 구현

#### Amazon Titan Embedding 모델 사용

- model_id='amazon.titan-embed-text-v1'
- 이 모델은 최대 512 Token 입력이 가능합니다. 추후에 최대 토큰 사이즈가 큰 모델이 나오면 바꾸어서 테스트하시기 바랍니니다.
- 일반적으로 한글 임베딩시에 512 토큰은 작습니다.

In [20]:
from langchain.embeddings import BedrockEmbeddings

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)
bedrock_embeddings

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f512bd65300>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

#### PyPDFDirectoryLoader 를 통한 PDF 파일 로딩

- chunk_size 는 임베딩 모델의 최대 입력 토큰이 512를 고려 해서 정했습니다.
- 아래 수치보다 증가시킬 경우에 Embedding Model 에 벡터 변환을 요구할 시에, 토큰이 512 보다 커서 에러가 발생합니다

In [21]:
#from langchain.indexes.vectorstore import VectorStoreIndexWrapper 에서 에러가 나면 sqlalchemy를 최신버전으로 업데이트 합니다.
#!pip install sqlalchemy==2.0.21

In [22]:
# 테스트 기준 sqlalchemy==2.0.21

import sqlalchemy
print(sqlalchemy.__version__)

2.0.21


In [23]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper


loader = PyPDFDirectoryLoader("rag_data_kr_pdf/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 230,
    chunk_overlap  = 50,
#    separators = ['\n']
#    separators = ['\n','\n\n']    
)
docs = text_splitter.split_documents(documents)

In [24]:
%%time
vectorstore_faiss_aws = FAISS.from_documents(
    documents=docs,
    embedding = bedrock_embeddings
)

print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)

vectorstore_faiss_aws: number of elements in the index=92::
CPU times: user 306 ms, sys: 22.6 ms, total: 329 ms
Wall time: 10.3 s


In [25]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 10 documents loaded is 1606 characters.
After the split we have 92 documents more than the original 10.
Average length among 92 documents (after split) is 216 characters.


In [26]:
print("docs[0].page_content: \n", docs[0].page_content)

docs[0].page_content: 
 CEO로서 두 번째 연례 주주 서한을 작성하기 위해 자리에 앉았을 때 저는 Amazon의 앞날에 대해 낙관적이고 활력을 얻었습니다. 2022년은 최근 기억에 있어 어려운 거시경제적 해 중 하나이고 우리 자체의 운영상의 어려움에도 불구하고 우리는 여전히 수요를 늘릴 방법을 찾았습니다(팬데믹 전반기에 경험한 전례 없는 성장에 더해). 우리는 장단기적으로 고객 경험을 의미 있게 개선하기 위해 대규모 사업을


In [27]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.68359375 -0.359375    0.27148438 ...  0.203125   -0.56640625
 -0.53125   ]
Size of the embedding:  (1536,)


#### 시맨틱 검색 (Semantic search)

LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 저장소를 쿼리하고 관련 문서를 반환할 수 있습니다. 뒤에서는 RetrievalQA 체인만 실행됩니다.


In [28]:
query = "아마존은 Generative AI 의 전략이 무엇인가요?"
print_ww(wrapper_store_faiss.query(query, llm=cl_llm))

 제가 이 편지의 전체 내용을 보지 않았기 때문에 아마존의 Generative AI 전략에 대해 확실히 말씀드리기 어렵습니다. 편지에서 Generative AI나 LLM에 대한
구체적인 전략이 언급된 부분이 없는 것 같습니다. 저는 이 질문에 대한 정확한 답변을 제공할 수 있는 충분한 정보가 없습니다. 죄송합니다. 아마존의 Generative AI 전략에
대한 더 많은 정보가 있다면, 그것을 바탕으로 답변을 제공할 수 있을 것 같습니다.


-----
의미론적 검색이 어떻게 작동하는지 살펴보겠습니다.

먼저 쿼리에 대한 임베딩 벡터를 계산하고
그런 다음 이 벡터를 사용하여 벡터 스토어에서 유사성 검색을 수행합니다.

In [29]:
v = bedrock_embeddings.embed_query(query)
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=3)
for r in results:
    print_ww(r.page_content)
    print('----')

[0.765625, -0.59375, -0.08300781, -0.29296875, 0.86328125, -0.19140625, 0.06542969, -0.00034332275, -1.2109375, -0.578125]
국제적으로 확장하고, 아직 아마존의 초기 단계인 대규모 소매 시장 부문을 추구하고, 판매자가 자신의 웹사이트에서 보다 효과적으로 판매할 수 있도록 돕기 위해 우리의 고유한 자산을
사용하는 것은 우리에게 어느 정도 자연스러운 확장입니다. 핵심 비즈니스에서 더 멀리 떨어져 있지만 고유한 기회가 있는 투자도 몇 가지 있습니다. 2003년에는 AWS가 전형적인
예였습니다. 2023년에는 Amazon
----
LLM과 GeneraTve AI가 그렇게 변혁적일 것이라고 생각하기 때문에 전체 편지를 쓸 수 있지만 향후 편지로 남겨두겠습니다. LLM과 GeneraTve AI가 고객, 주주 및
Amazon에게 큰 문제가 될 것이라고 가정해 보겠습니다.  그래서 마지막으로 저는 우리가 이 도전적인 거시경제 시대에 진입했을 때보다 더 강력한 위치에서 등장할 것이라고
낙관합니다. 여기에는 몇 가지 이유가 있으며 위에서 많은
----
잠재력이 있으며 기술 및 창의적 능력을 갖춘 회사가 상대적으로 적다는 점에서 AWS와 유사합니다. 그것을 추구하는 투자 가설로.  제가 언급할 마지막 투자 영역은 Amazon이
앞으로 수십 년 동안 모든 비즈니스 영역에서 발명할 수 있도록 설정하는 핵심이며 대규모 언어 모델("LLM") 및 생성 AI입니다. 머신 러닝은 수십 년 동안 유망한 기술이었지만
기업에서 널리 사용되기 시작한 것은 불과 5~10년
----


---

#### 메모리

모든 챗봇에는 사용 사례에 따라 맞춤화된 다양한 옵션을 갖춘 QA 체인이 필요합니다. 그러나 챗봇에서는 모델이 답변을 제공하기 위해 이를 고려할 수 있도록 항상 대화 기록을 보관해야 합니다. 이 예에서는 대화 기록을 유지하기 위해 ConversationBufferMemory와 함께 LangChain의 [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db)을 사용합니다.

출처: https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db

뒤에서 무슨 일이 일어나고 있는지 모두 보려면 'verbose'를 'True'로 설정하세요.

In [30]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


---

#### ConversationRetrievalChain에 사용되는 매개변수

* **retriever**: 우리는 `VectorStore`가 지원하는 `VectorStoreRetriever`를 사용했습니다. 텍스트를 검색하려면 `"similarity"` 또는 `"mmr"`라는 두 가지 검색 유형을 선택할 수 있습니다. `search_type="similarity"`는 질문 벡터와 가장 유사한 텍스트 청크 벡터를 선택하는 검색 객체에서 유사성 검색을 사용합니다.

* **메모리**: 이력을 저장하는 메모리 체인

* **dense_question_prompt**: 사용자의 질문이 주어지면 이전 대화와 해당 질문을 사용하여 독립형 질문을 구성합니다.

* **chain_type**: 채팅 기록이 길고 상황에 맞지 않는 경우 이 매개변수를 사용하고 옵션은 `stuff`, `refine`, `map_reduce`, `map-rerank`입니다.

질문이 컨텍스트 범위를 벗어나면 모델은 답을 모른다고 대답합니다.

**참고**: 체인이 어떻게 작동하는지 궁금하다면 `verbose=True` 줄의 주석 처리를 해제하세요.

In [31]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

채팅을 해보죠. 아래와 같은 질문을 해보세요. 
1. 아마존은 Generative AI 의 전략이 무엇인가요?

In [32]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

---

이는 이 노트북의 시작 부분에 설명된 것과 동일한 이유로 발생합니다. 기본 랭체인 프롬프트는 Claude에게 최적이 아닙니다. 다음 셀에서는 두 개의 새로운 프롬프트를 설정하겠습니다. 하나는 질문의 표현을 변경하기 위한 것이고, 다른 하나는 해당 질문의 답변을 얻기 위한 것입니다.


In [33]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

새로운 질문으로만 대답하세요.


Human: 이전 대화를 고려하여 질문을 어떻게 하시겠습니까?: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provide longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v1", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}


Human: <q></q> XML 태그 내의 질문에 답하려면 최대 3개의 문장을 사용하세요.

<q>{question}</q>

답변에 XML 태그를 사용하지 마세요. 답변이 context에 없으면 "죄송합니다. 문맥에서 답을 찾을 수 없어서 모르겠습니다."라고 말합니다.

Assistant:""")

다시 채팅을 해보죠. 아래와 같은 질문을 해보세요.

1. 아마존은 Generative AI 의 전략이 무엇인가요?

In [34]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

"아마존은 Generative AI 의 전략이 무엇인가요?" 라는 질문에 주주 서한 문서 기반으로 답변을 출력하는 것을 확인할 수 있습니다.

![rag-chatbot-result-01](./img/rag-chatbot-result-01.png)